# Type assertion

**Type assertion** - это операция, которую можно применить к значению интерфейсного типа. В общем виде эта операция выглядит как **x.(T)**, где x - это выражение интерфейсного типа, а T - некоторый тип(конкретный или интерфейсный). Type assertion работает с динамическим типом интерфейсного значения. Тут возможны два варианта:  
1. **T является конкретным типом**. В этом случае проверяется совпадает ли динамический тип x с типом T. Если совпадает, то результатом является динамическое значение x, которое конечно же имеет тип T. Другими словами, операция type assertion с конкретным типом извлекает конкретное значение из операнда:

In [1]:
import (
    "fmt"
    "io"
    "os"
)

var w io.Writer      // Интерфейсное значение.
w = os.Stdout        // Присваивание проходит, т.к. os.Stdout удовлетворяет интерфейсу io.Writer.
f := w.(*os.File)    // Type assertion проходит, т.к. динамический тип w это тип *os.File.
fmt.Sprintf("%T", f)

*os.File


 Если динамический тип x не равен T, то происходит **паника**:

In [ ]:
import "bytes"
// Следующий код вызовет панику.
c := w.(*bytes.Buffer)

2. **Т является интерфейсным типом**. В этом случае проверяется удовлетворяет ли динамический тип x интерфейсному типу T. Если удовлетворяет, то результатом операции является значение интерфейсного типа T, у которого динамические тип и значение точно такие же, как и у интерфейсного значения x. Другими словами, type assertion с интерфейсным типом изменяет тип выражения, возможно позволяя получить доступ к большему количеству методов. Если динамический тип x не удовлетворяет интерфейсу T, то происходит паника.

In [2]:
var w io.Writer
w = os.Stdout
// Динамический тип w равен *os.File и этот тип удовлетворяет интерфейсу io.ReadWriter.
// Именно поэтому работает следующий код.
rw := w.(io.ReadWriter)

Вне зависимости от того какой тип T используется в операции type assertion(конкретный или интерфейсный), если операнд(т.е. x) является nil'ом, то происходит паника:

In [ ]:
var w io.ReadWriter
var r io.Reader
r = w
// Следующая строка вызовет панику.
r = w.(io.Reader)

Для того чтобы не вызывать панику при вызове операции type assertion, можно воспользоваться второй переменной, указывающей на результат преобразования:

In [3]:
import "bytes"

var w io.Writer = os.Stdout
f, ok := w.(*os.File)
fmt.Println(ok)

// type assertion не проходит, но нет также и паники. b равен nil.
b, ok := w.(*bytes.Buffer)
fmt.Sprint(ok)

true
false


## Type switch

Для того чтобы не писать длинные цепочки конструкций вида if else при использовании операций type assertion можно воспользоваться специальным видом оператора switch, который называется **type switch**. В самом простом виде этот оператор состоит из операнда **x.(type)** - здесь type это буквально текст, который нужно набирать, а не какой-то другой тип, который нужно подставлять - и вариантов case для каждого интересующего нас типа. Оператор type switch позволяет нам определить несколько направлений ветвления в зависимости от динамического типа интерфейсного значения x:

In [4]:
func foo(x interface{}) string {
    switch x.(type) {
        case nil:
            return "nil"
        
        case int:
            return fmt.Sprintf("%d", x)
        
        case bool:
            // Каждый case это новый лексический блок, в котором переменная с таким
            // же названием как и в внешнем блоке, может перекрыть внешнюю.
            x := x.(bool)
            if x {
                return "TRUE"
            }
            return "FALSE"
        
        default:
            return "UNKNOWN"
    }
}

fmt.Println(foo(23))
fmt.Println(foo(true))
fmt.Sprint(foo(1.2))

23
TRUE
UNKNOWN


В этом примере в случае case bool после того как мы уже точно узнали, что динамический тип x равен bool, мы провели операцию type assertion, чтобы получить значение типа bool(в операторе if можно использовать только такие значения). Оператор type switch позволяет выполнить это действие один раз в самом начале и в каждом case'е операнд будет иметь такой тип, который используется в этом case'e. Но здесь есть и исключение: когда в case'е указано несколько типов, то этот case получает исходное интерфейсное значение:

In [5]:
func foo(x interface{}) string {
    switch x := x.(type) {
        case nil:
            return "nil"
        
        case int, uint:
            // Здесь x имеет тип interface{}.
            return fmt.Sprintf("%d", x)
        
        case bool:
            // x имеет тип bool.
            if x {
                return "TRUE"
            }
            return "FALSE"
        
        default:
            return "UNKNOWN"
    }
}

fmt.Println(foo(23))
fmt.Println(foo(true))
fmt.Sprint(foo(1.2))

23
TRUE
UNKNOWN
